In [1]:
import os
import time
import pandas as pd
import numpy as np
import networkx as nx
import collections
from scipy import sparse as sp
from scipy.stats import rankdata

import itertools
from itertools import combinations, combinations_with_replacement, cycle
from functools import reduce

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from util import *

import colorcet as cc

import bokeh
from bokeh.io import output_notebook, output_file, show, save
from bokeh.plotting import figure
from bokeh.models import (Rect, MultiLine, Circle, Span, Label,
                          GraphRenderer, StaticLayoutProvider,
                          NodesAndLinkedEdges,
                          HoverTool, TapTool, ColumnDataSource,
                          LinearColorMapper, LogColorMapper, CategoricalColorMapper,
                          CategoricalMarkerMapper,
                          BoxSelectTool,
                          ColorBar, BasicTicker, BoxZoomTool, FactorRange,
                          Range1d)
from bokeh.models import CategoricalTicker, FixedTicker, BoxAnnotation
from bokeh.models import Arrow, NormalHead, OpenHead, VeeHead, LabelSet

from bokeh.transform import transform, factor_cmap, linear_cmap, log_cmap
from bokeh.layouts import row, column, gridplot
output_notebook()

Loading BokehJS ...

In [2]:
def joint_marginal(df, c1, c2, include_fraction=False):
    """Given a dataframe and two columns, return a dataframe with the joint and marginal counts."""
    j = df.value_counts([c1, c2])
    j.name = "joint_count"
    j = j.reset_index()

    m1 = df.value_counts(c1)
    m1.name = f"{c1}_count"
    j = j.merge(m1, left_on=c1, right_index=True)

    m2 = df.value_counts(c2)
    m2.name = f"{c2}_count"
    j = j.merge(m2, left_on=c2, right_index=True)

    if include_fraction:
        j["joint_fraction"] = j["joint_count"] / j["joint_count"].sum()
        j[f"{c1}_fraction"] = j["joint_count"] / j[f"{c1}_count"]
        j[f"{c2}_fraction"] = j["joint_count"] / j[f"{c2}_count"]
    return j


In [22]:
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "oviIN/clustering_" + hemibrain_version
hemibrain_nodes = "inputsoutputs_key.txt"
hemibrain_edges = "inputsoutputs.txt"
#hemibrain_nodes = "only_inputs_key.txt"
#hemibrain_edges = "only_inputs.txt"
#hemibrain_nodes = "only_outputs_key.txt"
#hemibrain_edges = "only_outputs.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

2023 07 07 15:01:10  Hemibrain data set being used: v1.2.1
2023 07 07 15:01:10  Set up directory info and useful lists


In [23]:
from neuprint import Client
from neuprint import fetch_roi_hierarchy


auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

2023 07 07 15:01:13  neuprint Client set up as `np_Client`, version 0.1.0
2023 07 07 15:01:13  Loading node dataframe
2023 07 07 15:01:13    (without centroids)
2023 07 07 15:01:14  Adding 'type group'
2023 07 07 15:01:14  Done!
2023 07 07 15:01:14  Loading directed edges from csv
2023 07 07 15:01:14  Done!
2023 07 07 15:01:14  Merging in cell info to edge df
2023 07 07 15:01:14  Done!


In [29]:
HB_node_df[HB_node_df['celltype']=='FS1A']

,key,0.0,0.05,0.1,0.5,0.75,1.0,instance,celltype,pre,...,cropped,statusLabel,cellBodyFiber,somaRadius,somaLocation,roiInfo,notes,inputRois,outputRois,type_group
id,,,,,,,,,,,,,,,,,,,,,
1014102651,47,3,5,5,9,9,43,FS1A(FQ9)_C2_L,FS1A,89,...,False,Roughly traced,PDM01,301.0,"[26807, 16994, 11456]","{'SNP(R)': {'pre': 45, 'post': 34, 'downstream...",FB2-6SMPSMP,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...","['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...",FS
1014223948,51,3,5,5,9,12,43,FS1A(FQ9)_C2_R,FS1A,87,...,False,Roughly traced,PDM04,291.5,"[18950, 10470, 18776]","{'SNP(L)': {'pre': 36, 'post': 43, 'downstream...",FB2-6SMPSMP,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FB-col...","['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'SMP(L)', 'S...",FS
1014461486,54,3,5,5,9,9,21,FS1A(FQ9)_C3_L,FS1A,92,...,False,Roughly traced,PDM01,291.0,"[28899, 16232, 12088]","{'SNP(R)': {'pre': 37, 'post': 23, 'downstream...",FB2-6SMPSMP,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...",FS
1014465452,55,3,5,5,9,12,21,FS1A(FQ9)_C3_L,FS1A,101,...,False,Roughly traced,PDM01,291.0,"[28255, 16569, 12008]","{'SNP(L)': {'pre': 25, 'post': 28, 'downstream...",FB2-6SMPSMP,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...",FS
1014465588,56,3,5,5,9,12,43,FS1A(FQ9)_C9_L,FS1A,96,...,False,Roughly traced,PDM04,245.0,"[33446, 11602, 19792]","{'SNP(R)': {'pre': 49, 'post': 31, 'downstream...",FB2-6SMPSMP,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...",FS
1014465666,57,3,5,5,12,12,49,FS1A(FQ9)_C9_L,FS1A,96,...,False,Roughly traced,PDM04,300.0,"[33748, 11191, 19440]","{'SNP(R)': {'pre': 45, 'post': 37, 'downstream...",FB2-6SMPSMP,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FBl2',...","['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FBl4',...",FS
1014474192,59,3,5,5,9,9,14,FS1A(FQ9)_C2_L,FS1A,112,...,False,Roughly traced,PDM01,301.0,"[27300, 17574, 10448]","{'SNP(L)': {'pre': 31, 'post': 18, 'downstream...",FB2-6SMPSMP,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...",FS
1039063931,106,3,5,5,9,12,90,FS1A(FQ9)_C5_R,FS1A,82,...,False,Roughly traced,PDM02,301.0,"[19582, 14276, 13168]","{'SNP(R)': {'pre': 28, 'post': 21, 'downstream...",FB2-6SMPSMP,"['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...","['CRE(-ROB,-RUB)(R)', 'CRE(-RUB)(L)', 'CRE(L)'...",FS
1039085109,110,3,5,5,12,12,43,FS1A(FQ9)_C6_R,FS1A,97,...,False,Roughly traced,PDM02,301.0,"[22191, 15842, 11760]","{'CX': {'post': 795, 'upstream': 795, 'mito': ...",FB2-6SMPSMP,"['CRE(-RUB)(L)', 'CRE(L)', 'CX', 'FB', 'FBl2',...","['CRE(-RUB)(L)', 'CRE(L)', 'INP', 'SMP(L)', 'S...",FS


In [17]:
chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

g = gridplot(fs, ncols=2)
show(g)

,0.0,0.05,joint_count,0.0_count,0.05_count,joint_fraction,0.0_fraction,0.05_fraction
0,4,5,597,778,602,0.173597,0.767352,0.991694
37,5,5,3,566,602,0.000872,0.005300,0.004983
52,6,5,2,550,602,0.000582,0.003636,0.003322
8,4,3,50,778,222,0.014539,0.064267,0.225225
9,1,3,46,712,222,0.013376,0.064607,0.207207
...,...,...,...,...,...,...,...,...
125,6,23,1,550,1,0.000291,0.001818,1.000000
126,6,26,1,550,1,0.000291,0.001818,1.000000
127,6,27,1,550,1,0.000291,0.001818,1.000000
208,6,159,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.05: 160


,0.0,0.1,joint_count,0.0_count,0.1_count,joint_fraction,0.0_fraction,0.1_fraction
0,4,5,621,778,632,0.180576,0.798201,0.982595
36,2,5,4,601,632,0.001163,0.006656,0.006329
54,5,5,2,566,632,0.000582,0.003534,0.003165
41,6,5,4,550,632,0.001163,0.007273,0.006329
255,3,5,1,232,632,0.000291,0.004310,0.001582
...,...,...,...,...,...,...,...,...
138,6,177,1,550,1,0.000291,0.001818,1.000000
141,6,18,1,550,1,0.000291,0.001818,1.000000
143,6,13,1,550,1,0.000291,0.001818,1.000000
263,6,201,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.1: 205


,0.0,0.5,joint_count,0.0_count,0.5_count,joint_fraction,0.0_fraction,0.5_fraction
0,5,15,136,566,157,0.039546,0.240283,0.866242
38,4,15,13,778,157,0.003780,0.016710,0.082803
694,6,15,1,550,157,0.000291,0.001818,0.006369
170,1,15,4,712,157,0.001163,0.005618,0.025478
284,2,15,3,601,157,0.000872,0.004992,0.019108
...,...,...,...,...,...,...,...,...
874,2,508,1,601,1,0.000291,0.001664,1.000000
970,2,163,1,601,1,0.000291,0.001664,1.000000
971,2,181,1,601,1,0.000291,0.001664,1.000000
975,2,191,1,601,1,0.000291,0.001664,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.5: 745


,0.0,0.75,joint_count,0.0_count,0.75_count,joint_fraction,0.0_fraction,0.75_fraction
0,3,5,53,232,53,0.015411,0.228448,1.000000
3,3,93,21,232,21,0.006106,0.090517,1.000000
6,3,95,17,232,17,0.004943,0.073276,1.000000
9,3,18,16,232,16,0.004653,0.068966,1.000000
15,3,52,13,232,14,0.003780,0.056034,0.928571
...,...,...,...,...,...,...,...,...
1038,6,910,1,550,1,0.000291,0.001818,1.000000
1040,6,909,1,550,1,0.000291,0.001818,1.000000
1044,6,886,1,550,1,0.000291,0.001818,1.000000
1047,6,879,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.75: 1098


In [7]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison.html")
save(g, title='Joint Marginal Analysis (full)')

'/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison.html'

In [8]:
# Inputs
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed_inputs-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "oviIN/clustering_inputs_" + hemibrain_version
hemibrain_nodes = "only_inputs_key.txt"
hemibrain_edges = "only_inputs.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

k = gridplot(fs, ncols=2)

2023 07 07 14:29:49  Hemibrain data set being used: v1.2.1
2023 07 07 14:29:49  Set up directory info and useful lists
2023 07 07 14:29:49  neuprint Client set up as `np_Client`, version 0.1.0
2023 07 07 14:29:49  Loading node dataframe
2023 07 07 14:29:49    (without centroids)
2023 07 07 14:29:49  Adding 'type group'
2023 07 07 14:29:49  Done!
2023 07 07 14:29:49  Loading directed edges from csv
2023 07 07 14:29:49  Done!
2023 07 07 14:29:49  Merging in cell info to edge df
2023 07 07 14:29:49  Done!


,0.0,0.05,joint_count,0.0_count,0.05_count,joint_fraction,0.0_fraction,0.05_fraction
0,1,13,374,519,392,0.148945,0.720617,0.954082
16,5,13,13,387,392,0.005177,0.033592,0.033163
31,4,13,3,302,392,0.001195,0.009934,0.007653
41,2,13,2,716,392,0.000796,0.002793,0.005102
9,1,1,57,519,271,0.022700,0.109827,0.210332
...,...,...,...,...,...,...,...,...
132,2,39,1,716,1,0.000398,0.001397,1.000000
133,2,38,1,716,1,0.000398,0.001397,1.000000
134,2,37,1,716,1,0.000398,0.001397,1.000000
135,2,36,1,716,1,0.000398,0.001397,1.000000


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.05: 101


,0.0,0.1,joint_count,0.0_count,0.1_count,joint_fraction,0.0_fraction,0.1_fraction
0,5,20,283,387,288,0.112704,0.731266,0.982639
31,3,20,5,587,288,0.001991,0.008518,0.017361
14,5,8,27,387,221,0.010753,0.069767,0.122172
33,4,8,4,302,221,0.001593,0.013245,0.018100
101,1,8,1,519,221,0.000398,0.001927,0.004525
...,...,...,...,...,...,...,...,...
61,3,18,1,587,1,0.000398,0.001704,1.000000
65,3,26,1,587,1,0.000398,0.001704,1.000000
68,3,74,1,587,1,0.000398,0.001704,1.000000
69,3,82,1,587,1,0.000398,0.001704,1.000000


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.1: 89


,0.0,0.5,joint_count,0.0_count,0.5_count,joint_fraction,0.0_fraction,0.5_fraction
0,3,5,150,587,151,0.059737,0.255537,0.993377
507,2,5,1,716,151,0.000398,0.001397,0.006623
2,3,10,46,587,46,0.018319,0.078365,1.000000
18,3,19,19,587,19,0.007567,0.032368,1.000000
19,3,263,18,587,18,0.007168,0.030664,1.000000
...,...,...,...,...,...,...,...,...
465,4,458,1,302,1,0.000398,0.003311,1.000000
467,4,462,1,302,1,0.000398,0.003311,1.000000
477,4,466,1,302,1,0.000398,0.003311,1.000000
479,4,480,1,302,1,0.000398,0.003311,1.000000


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.5: 533


,0.0,0.75,joint_count,0.0_count,0.75_count,joint_fraction,0.0_fraction,0.75_fraction
0,3,5,129,587,131,0.051374,0.219761,0.984733
493,2,5,2,716,131,0.000796,0.002793,0.015267
3,3,10,24,587,24,0.009558,0.040886,1.000000
8,3,309,14,587,14,0.005575,0.023850,1.000000
17,3,19,10,587,10,0.003982,0.017036,1.000000
...,...,...,...,...,...,...,...,...
720,4,717,1,302,1,0.000398,0.003311,1.000000
721,4,716,1,302,1,0.000398,0.003311,1.000000
748,4,225,1,302,1,0.000398,0.003311,1.000000
920,4,279,1,302,1,0.000398,0.003311,1.000000


Clusters found at chi = 0.0: 5
Clusters found at chi = 0.75: 780


In [9]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_inputs.html")
save(k, title='Joint Marginal Analysis (Inputs)')

'/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_inputs.html'

In [10]:
# outputs
hemibrain_version = "v1.2.1"
log_msg("Hemibrain data set being used:", hemibrain_version)

preproc_dir = "oviIN/preprocessed_outputs-" + hemibrain_version
preproc_nodes = "preprocessed_nodes.csv"
preproc_centroids = "x"
preproc_edges = "preprocessed_undirected_edges.csv"

hemibrain_dir = "oviIN/clustering_outputs_" + hemibrain_version
hemibrain_nodes = "only_outputs_key.txt"
hemibrain_edges = "only_outputs.txt"

figure_dir = os.path.join("figures","paper")
movie_dir = os.path.join("movies")
analysis_dir = os.path.join("analysis",hemibrain_version)
obj_dir = os.path.join("obj",hemibrain_version)  # 3d objects from, e.g. fetch_roi_mesh
skel_dir = os.path.join("skeleton", hemibrain_version)  # skeleta of neurons in .csv format


for d in [figure_dir, analysis_dir, obj_dir, movie_dir]:
    if not os.path.isdir(d):
        log_msg("Creating directory", d)
        os.makedirs(d)

reneel_params = list(sorted(['0.05','0.1','0.5','0.75'], key=float))
type_params = ['celltype','instance']
list_of_params = reneel_params + type_params

log_msg("Set up directory info and useful lists")

auth_token_file = open("flybrain.auth.txt", 'r')
auth_token = next(auth_token_file).strip()
try:
    np_client = Client('neuprint.janelia.org', dataset='hemibrain:' + hemibrain_version, token=auth_token)
    log_msg("neuprint Client set up as `np_Client`, version", np_client.fetch_version())
except:
    np_client = None
    log_msg("neuprint Client set up failed!")

log_msg("Loading node dataframe")
if os.path.isfile(os.path.join(preproc_dir, preproc_centroids)):
    log_msg("  (with centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_centroids), index_col=0)
else:
    log_msg("  (without centroids)")
    HB_node_df = pd.read_csv(os.path.join(preproc_dir, preproc_nodes), index_col=0)
log_msg("Adding 'type group'")
HB_node_df["type_group"] = HB_node_df["celltype"].apply(simplify_type)
log_msg("Done!")

log_msg("Loading directed edges from csv")
HB_edge_df = pd.read_csv(os.path.join(hemibrain_dir, hemibrain_edges), delimiter=' ', header=None).rename(columns={0: "pre", 1:"post"})
log_msg("Done!")

log_msg("Merging in cell info to edge df")
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='pre', right_index=True)
HB_edge_df = HB_edge_df.merge(HB_node_df[list_of_params + ['type_group']], left_on='post', right_index=True, suffixes=['pre', 'post'])
log_msg("Done!")

chi1 = '0.0'
fs=[]
for chi in reneel_params:
   jm = joint_marginal(HB_node_df, chi1, chi, include_fraction=True)
   display(jm)
   print(f"Clusters found at chi = {chi1}:", jm[chi1].max())
   print(f"Clusters found at chi = {chi}:", jm[chi].max())

   # sort the clusters on the y axis to get a more "diagonal" plot
   yrange = jm.sort_values([f"{chi}_fraction"], ascending=False).groupby(chi).agg({chi1: "first", f"{chi}_fraction": "first", "joint_count": "first"}).sort_values([chi1, "joint_count"], ascending=[True, False]).index
   f = figure(title=f"Clusters at chi = {chi} vs. clusters at chi = {chi1}",
            x_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.0"].max())]),
         #    y_range=FactorRange(factors=[str(i + 1) for i in range(jm["0.05"].max())]),
            y_range=FactorRange(factors=[str(y) for y in yrange]),
            width=600, height=1000)


   jm["x"] = jm[chi1].apply(str)  # bokeh factor range has to have strings, so we have to convert these
   jm["y"] = jm[chi].apply(str)

   #fig_kws = dict(title=None,border_fill_color=None,outline_line_color=None, background_fill_color=None,)
   #f = figure(**fig_kws)

   f.rect(x="x", y="y",
         width=f"{chi1}_fraction", height=f"{chi}_fraction",
         source=jm)
   f.add_tools(HoverTool(tooltips={"Neurons": "@joint_count (@joint_fraction{%%} of Hemibrain)",
                                 f"Fraction of {chi1}": f"@{{{chi1}_fraction}}{{2.%%}}",
                                 f"Fraction of {chi}": f"@{{{chi}_fraction}}{{2.%%}}"}))
   fs.append(f)

i = gridplot(fs, ncols=2)

2023 07 07 14:30:15  Hemibrain data set being used: v1.2.1
2023 07 07 14:30:15  Set up directory info and useful lists


2023 07 07 14:30:16  neuprint Client set up as `np_Client`, version 0.1.0
2023 07 07 14:30:16  Loading node dataframe
2023 07 07 14:30:16    (without centroids)
2023 07 07 14:30:16  Adding 'type group'
2023 07 07 14:30:16  Done!
2023 07 07 14:30:16  Loading directed edges from csv
2023 07 07 14:30:16  Done!
2023 07 07 14:30:16  Merging in cell info to edge df
2023 07 07 14:30:16  Done!


,0.0,0.05,joint_count,0.0_count,0.05_count,joint_fraction,0.0_fraction,0.05_fraction
0,4,5,597,778,602,0.173597,0.767352,0.991694
37,5,5,3,566,602,0.000872,0.005300,0.004983
52,6,5,2,550,602,0.000582,0.003636,0.003322
8,4,3,50,778,222,0.014539,0.064267,0.225225
9,1,3,46,712,222,0.013376,0.064607,0.207207
...,...,...,...,...,...,...,...,...
125,6,23,1,550,1,0.000291,0.001818,1.000000
126,6,26,1,550,1,0.000291,0.001818,1.000000
127,6,27,1,550,1,0.000291,0.001818,1.000000
208,6,159,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.05: 160


,0.0,0.1,joint_count,0.0_count,0.1_count,joint_fraction,0.0_fraction,0.1_fraction
0,4,5,621,778,632,0.180576,0.798201,0.982595
36,2,5,4,601,632,0.001163,0.006656,0.006329
54,5,5,2,566,632,0.000582,0.003534,0.003165
41,6,5,4,550,632,0.001163,0.007273,0.006329
255,3,5,1,232,632,0.000291,0.004310,0.001582
...,...,...,...,...,...,...,...,...
138,6,177,1,550,1,0.000291,0.001818,1.000000
141,6,18,1,550,1,0.000291,0.001818,1.000000
143,6,13,1,550,1,0.000291,0.001818,1.000000
263,6,201,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.1: 205


,0.0,0.5,joint_count,0.0_count,0.5_count,joint_fraction,0.0_fraction,0.5_fraction
0,5,15,136,566,157,0.039546,0.240283,0.866242
38,4,15,13,778,157,0.003780,0.016710,0.082803
694,6,15,1,550,157,0.000291,0.001818,0.006369
170,1,15,4,712,157,0.001163,0.005618,0.025478
284,2,15,3,601,157,0.000872,0.004992,0.019108
...,...,...,...,...,...,...,...,...
874,2,508,1,601,1,0.000291,0.001664,1.000000
970,2,163,1,601,1,0.000291,0.001664,1.000000
971,2,181,1,601,1,0.000291,0.001664,1.000000
975,2,191,1,601,1,0.000291,0.001664,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.5: 745


,0.0,0.75,joint_count,0.0_count,0.75_count,joint_fraction,0.0_fraction,0.75_fraction
0,3,5,53,232,53,0.015411,0.228448,1.000000
3,3,93,21,232,21,0.006106,0.090517,1.000000
6,3,95,17,232,17,0.004943,0.073276,1.000000
9,3,18,16,232,16,0.004653,0.068966,1.000000
15,3,52,13,232,14,0.003780,0.056034,0.928571
...,...,...,...,...,...,...,...,...
1038,6,910,1,550,1,0.000291,0.001818,1.000000
1040,6,909,1,550,1,0.000291,0.001818,1.000000
1044,6,886,1,550,1,0.000291,0.001818,1.000000
1047,6,879,1,550,1,0.000291,0.001818,1.000000


Clusters found at chi = 0.0: 6
Clusters found at chi = 0.75: 1098


In [11]:
output_file("/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_outputs.html")
save(i, title='Join Marginal Analysis (Outputs)')

'/Users/rhessa/flybrain-clustering/Figures-oviIN/joint_marginal_comparison_outputs.html'